<a href="https://colab.research.google.com/github/poojanalan10/OregonHighway/blob/master/Data_Integration_Activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [246]:
import pandas as pd
import datetime

ACSdata = pd.read_csv('/content/acs2017_census_tract_data.csv.zip')
CovidData = pd.read_csv('/content/COVID_county_data 2.csv.zip')

# ACSdata.head(10)
# CovidData.head(10)

A. Aggregate Census Data to County Level
Your integration will use two different dimensions: location (as indicated by state and county) and time. You should greatly simplify your processing and reduce your time by pre-processing your data along each of these dimensions.

The ACS data is separated into “Census Tracts” which are regions within counties that correspond to groups of approximately 4000 people. The Census Bureau defines these to help organize the actual job of collecting census data, but this grouping can make your Data Engineering job more more challenging. This level of detail is not needed for your county-level analysis, and you can greatly decrease your efforts by aggregating per-tract data to the county level.  

Create a python program that produces a one-row-per-county version of the ACS data set. To do this you will need to think about how to properly aggregate Census Tract-level data into County-level summaries. 

In this step you can also eliminate unneeded columns from the ACS data. 

Question: Show your aggregated county-level data rows for the following counties: Loudon County Virginia, Washington County Oregon, Harlan County Kentucky, Malheur County Oregon



In [247]:
#integratedData = pd.concat([ACSdata,CovidData], axis = 1)

#storing the county and it's corresponding state data and concatenating all the county,state pair
countyLoudounData = ACSdata[(ACSdata['County'] == 'Loudoun County') & (ACSdata['State'] == 'Virginia')]
countyHarlanData = ACSdata[(ACSdata['County'] == 'Washington County') & (ACSdata['State'] == 'Oregon')]
countyWashingtonData = ACSdata[(ACSdata['County'] == 'Harlan County') & (ACSdata['State'] == 'Kentucky')]
countyMalheurData = ACSdata[(ACSdata['County'] == 'Malheur County') & (ACSdata['State'] == 'Oregon')]
countyData = pd.concat([countyLoudounData,countyHarlanData,countyWashingtonData,countyMalheurData])
#countyData = countyData.groupby(["County","State"]).sum()
countyData['County'] = countyData['County'].map(lambda x: x.rstrip('County'))
countyData['Poverty'] = (countyData['Poverty']/100) * countyData['TotalPop']
countyData['IncomePerCap'] = (countyData['IncomePerCap']) * countyData['TotalPop']
countyData = countyData[['County','State','TotalPop','Poverty','IncomePerCap']]

#print(countyData)
#countyData = countyData.groupby(["County","State"]).sum()
#countyData = countyData.groupby(["County","State"], as_index=False).agg("sum")
countyData = countyData.dropna()
countyData = countyData.groupby(["County","State"], as_index=False).agg(
     Population  = pd.NamedAgg(column="TotalPop", aggfunc = "sum"),
     Poverty  =  pd.NamedAgg(column="Poverty", aggfunc = "sum"),
     PerCapitaIncome =  pd.NamedAgg(column="IncomePerCap", aggfunc = "sum")
      )
countyData["Poverty"] = (countyData['Poverty']*100)/countyData['Population']
countyData["PerCapitaIncome"] = (countyData['PerCapitaIncome'])/countyData['Population']
print(countyData)


        County     State  Population    Poverty  PerCapitaIncome
0      Harlan   Kentucky       27548  35.669482     15456.971032
1     Loudoun   Virginia      374558   3.689598     50455.645745
2     Malheur     Oregon       30421  24.298225     17567.504323
3  Washington     Oregon      572071  10.321202     35369.047499


B. Simplify the COVID Data
You can simplify the COVID data along the time dimension. The COVID data set contains day-level resolution data from (approximately) March of 2020 through February of 2021. However, you will only need four data points per county: total cases, total deaths, cases reported during December of 2020 and deaths reported during December 2020. 

Create a python program that reduces the COVID data to one line per county. 

Question: Show your simplified COVID data for the counties listed above. 


In [248]:
#December2020
start_date = "01122020"
end_date = "31122020"
format_str = '%d%m%Y' # The format The date - 29 Dec 2017 is 29122017
datetime_obj_start_date = datetime.datetime.strptime(start_date, format_str)
datetime_obj_end_date = datetime.datetime.strptime(end_date, format_str)
# print(datetime_obj_start_date.date())
# print(datetime_obj_end_date.date())
dateDecemberData = []
format_str_frame = '%y-%m-%d'
#filtering data to accomodate the data for the month of December, 2020
after_start_date = CovidData["date"] >= "2020-12-01"
before_end_date = CovidData["date"] <= "2020-12-31"
between_two_dates = after_start_date & before_end_date
covidDataDecember = CovidData.loc[between_two_dates]

#dropping date to aggregate data accroding to county 
covidDataDecember = covidDataDecember.drop(["date"],axis = 1)

#filtering by the counties mentioned above for the month of December
countyLoudounDecemberData = covidDataDecember[(covidDataDecember['county'] == 'Loudoun') & (covidDataDecember['state'] == 'Virginia')]
countyHarlanDecemberData = covidDataDecember[(covidDataDecember['county'] == 'Washington') & (covidDataDecember['state'] == 'Oregon')]
countyWashingtonDecemberData = covidDataDecember[(covidDataDecember['county'] == 'Harlan') & (covidDataDecember['state'] == 'Kentucky')]
countyMalheurDecemberData = covidDataDecember[(covidDataDecember['county'] == 'Malheur') & (covidDataDecember['state'] == 'Oregon')]
countyDatacovidDataDecember = pd.concat([countyLoudounDecemberData,countyHarlanDecemberData,countyWashingtonDecemberData,countyMalheurDecemberData])

#filtering by the counties mentioned above for till date
countyLoudounData = CovidData[(CovidData['county'] == 'Loudoun') & (CovidData['state'] == 'Virginia')]
countyHarlanData = CovidData[(CovidData['county'] == 'Washington') & (CovidData['state'] == 'Oregon')]
countyWashingtonData = CovidData[(CovidData['county'] == 'Harlan') & (CovidData['state'] == 'Kentucky')]
countyMalheurData = CovidData[(CovidData['county'] == 'Malheur') & (CovidData['state'] == 'Oregon')]
countyDatacovidDatatillDate = pd.concat([countyLoudounData,countyHarlanData,countyWashingtonData,countyMalheurData])

#grouping by county and state and calculating the total number of covid cases and deaths recorded till date
covidReporttoDate = countyDatacovidDatatillDate.groupby(["county","state"], as_index=False).agg(
    TotalCases   = pd.NamedAgg(column="cases",aggfunc = "max"),
    TotalDeaths  = pd.NamedAgg(column="deaths",aggfunc = "max")
    )

#grouping by county and state and getting the sum of deaths and no of covid cases in the month of December 2020
#per county
#using @as_index=False allows the columns to be retained as an index
covidReport = countyDatacovidDataDecember.groupby(["county","state"], as_index=False).agg(
    Dec2020Cases = pd.NamedAgg(column="cases", aggfunc = "max"),
    Dec2020Deaths = pd.NamedAgg(column="deaths",aggfunc = "max")
    )
#A different approach to group by for a single column
# covidReport = covidDataDecember.groupby(["county","state"])["cases"].sum().to_frame("cases").reset_index()
covidReport["TotalDeaths"] = covidReporttoDate["TotalDeaths"]
covidReport["TotalCases"] = covidReporttoDate["TotalCases"]
covidReport = covidReport[['county','state','TotalCases','Dec2020Cases','TotalDeaths','Dec2020Deaths']]
print(covidReport)

       county     state  TotalCases  Dec2020Cases  TotalDeaths  Dec2020Deaths
0      Harlan  Kentucky        2352          1538         68.0           18.0
1     Loudoun  Virginia       22557         14169        199.0          159.0
2     Malheur    Oregon        3331          2914         58.0           50.0
3  Washington    Oregon       20866         16070        209.0          142.0


C. Integrate COVID Data with ACS Data
Create a single pandas DataFrame containing one row per county and using the columns described above. You are free to add additional columns if needed. For example, you might want to normalize all of the COVID data by the population of each county so that you have a consistent “number of cases/deaths per 100000 residents” value for each county.

Question: List your integrated data for all counties in the State of Oregon.



In [249]:
covidReportfiltered = covidReport[['TotalCases','Dec2020Cases','TotalDeaths','Dec2020Deaths']]
integratedData = pd.concat([countyData,covidReportfiltered], axis=1)
integratedData['cases per 100,000 res'] = (integratedData['TotalCases']*100000)/integratedData['Population']
integratedData['death per 100,000 res'] = (integratedData['TotalDeaths']*100000)/integratedData['Population']

print(integratedData)

        County     State  Population    Poverty  PerCapitaIncome  TotalCases  \
0      Harlan   Kentucky       27548  35.669482     15456.971032        2352   
1     Loudoun   Virginia      374558   3.689598     50455.645745       22557   
2     Malheur     Oregon       30421  24.298225     17567.504323        3331   
3  Washington     Oregon      572071  10.321202     35369.047499       20866   

   Dec2020Cases  TotalDeaths  Dec2020Deaths  cases per 100,000 res  \
0          1538         68.0           18.0            8537.824887   
1         14169        199.0          159.0            6022.298282   
2          2914         58.0           50.0           10949.672923   
3         16070        209.0          142.0            3647.449355   

   death per 100,000 res  
0             246.841876  
1              53.129288  
2             190.657769  
3              36.533927  


D. Analysis
For each of the following, determine the strength of the correlation between each pair of variables. Compute the correlation strength by calculating the Pearson correlation coefficient R for pairs of columns in your DataFrame. For example, if you have a DataFrame df with each row representing a distinct county, and columns named ‘TotalCases’ and ‘Poverty’, then you can compute R like this:

R = df[‘TotalCases’].corr(df[‘Poverty’])

For any R that is > 0.5 or < -0.5 also display a scatter plot (see pandas scatterplot and seaborn documentation for information about how to display scatter plots from DataFrame data).

The COVID numbers should be normalized to population (# of cases per 100,000 residents) so that different sized counties are comparable. So for example, “COVID total cases” below really means “((COVID total cases in county * 100000) / population of county)”.

In [250]:

#All counties across USA
#ACSdata['County'] = ACSdata['County'].map(lambda x: x.rstrip(' County'))
ACSdata['Poverty'] = (ACSdata['Poverty']/100) * ACSdata['TotalPop']
ACSdata['IncomePerCap'] = (ACSdata['IncomePerCap']) * ACSdata['TotalPop']
ACSdata = ACSdata[['County','State','TotalPop','Poverty','IncomePerCap']]
ACSdata = ACSdata.dropna()
ACSdata = ACSdata.groupby(["County","State"], as_index=False).agg(
     Population  = pd.NamedAgg(column="TotalPop", aggfunc = "sum"),
     Poverty  =  pd.NamedAgg(column="Poverty", aggfunc = "sum"),
     PerCapitaIncome =  pd.NamedAgg(column="IncomePerCap", aggfunc = "sum")
      )
ACSdata["Poverty"] = (ACSdata['Poverty']*100)/ACSdata['Population']
ACSdata["PerCapitaIncome"] = (ACSdata['PerCapitaIncome'])/ACSdata['Population']
ACSdata = ACSdata.rename(columns={'State':'state','County':'county'})
print(len(ACSdata))
ACSdata.head(5)

3220


,county,state,Population,Poverty,PerCapitaIncome
0,Abbeville County,South Carolina,24788,22.663628,19234.247176
1,Acadia Parish,Louisiana,62607,21.630767,21590.908876
2,Accomack County,Virginia,32840,19.905271,24265.653959
3,Ada County,Idaho,435117,11.909603,31642.314649
4,Adair County,Iowa,7192,9.479060,28860.857620


In [251]:
#All counties across USA COVID Data

#grouping by county and state and calculating the total number of covid cases and deaths recorded till date
covidReporttoDateAllCounties = CovidData.groupby(["county","state"], as_index=False).agg(
    TotalCases   = pd.NamedAgg(column="cases",aggfunc = "max"),
    TotalDeaths  = pd.NamedAgg(column="deaths",aggfunc = "max")
    )
#grouping by county and state and getting the sum of deaths and no of covid cases in the month of December 2020
#per county
#using @as_index=False allows the columns to be retained as an index
covidReportAllCounties = covidDataDecember.groupby(["county","state"], as_index=False).agg(
    Dec2020Cases = pd.NamedAgg(column="cases", aggfunc = "max"),
    Dec2020Deaths = pd.NamedAgg(column="deaths",aggfunc = "max")
    )
covidReportAllCounties["TotalDeaths"] = covidReporttoDateAllCounties["TotalDeaths"]
covidReportAllCounties["TotalCases"] = covidReporttoDateAllCounties["TotalCases"]
covidReportAllCounties['county'] = covidReportAllCounties['county'].astype(str) + ' County'


covidReportAllCounties = covidReportAllCounties[['county','state','TotalCases','Dec2020Cases','TotalDeaths','Dec2020Deaths']]
#covidReportAllCounties = covidReportAllCounties.drop(columns=['county','state'])
print("Length of covidReportAllCounties")
print(len(covidReportAllCounties))

covidReportAllCounties.head(5)

Length of covidReportAllCounties
3247


,county,state,TotalCases,Dec2020Cases,TotalDeaths,Dec2020Deaths
0,Abbeville County,South Carolina,2174,1275,34.0,25.0
1,Acadia County,Louisiana,5999,5082,181.0,153.0
2,Accomack County,Virginia,2552,1698,35.0,27.0
3,Ada County,Idaho,46190,38417,428.0,355.0
4,Adair County,Iowa,790,606,26.0,17.0


In [264]:
#Integrated Data for USA
pd.options.display.max_columns = None
pd.options.display.max_rows = None

integrated_df=covidReportAllCounties.merge(ACSdata, on=['county','state'], how='inner')
integrated_df['TotalCases'] = (integrated_df['TotalCases']*100000)/integrated_df['Population']
integrated_df['TotalDeaths'] = (integrated_df['TotalDeaths']*100000)/integrated_df['Population']
integrated_df['Dec2020Cases'] = (integrated_df['Dec2020Cases']*100000)/integrated_df['Population']
integrated_df['Dec2020Deaths'] = (integrated_df['Dec2020Deaths']*100000)/integrated_df['Population']
integrated_df.head(24)
print("Length of integrated_df")
print(len(integrated_df))
# integrated_df.to_csv('data.csv',index=False)
# from google.colab import files
# files.download("data.csv")

integrated_df.head()

Length of integrated_df
3002


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,county,state,TotalCases,Dec2020Cases,TotalDeaths,Dec2020Deaths,Population,Poverty,PerCapitaIncome
0,Abbeville County,South Carolina,8770.372761,5143.617880,137.163143,100.855253,24788,22.663628,19234.247176
1,Accomack County,Virginia,7771.010962,5170.523752,106.577345,82.216809,32840,19.905271,24265.653959
2,Ada County,Idaho,10615.535592,8829.119524,98.364348,81.587251,435117,11.909603,31642.314649
3,Adair County,Iowa,10984.427141,8426.028921,361.512792,236.373749,7192,9.479060,28860.857620
4,Adair County,Kentucky,8526.730211,6703.273933,253.833402,233.112308,19304,21.451083,18408.087857


In [281]:
Oregondata = integrated_df[integrated_df["state"] == "Oregon"]
Oregondata.head()

,county,state,TotalCases,Dec2020Cases,TotalDeaths,Dec2020Deaths,Population,Poverty,PerCapitaIncome
109,Baker County,Oregon,3936.170213,2953.692115,43.804756,31.289111,15980,15.083855,25820.273154
175,Benton County,Oregon,2547.337647,1526.362905,18.130517,12.464730,88249,22.421152,30872.824361
486,Clackamas County,Oregon,3299.313435,2514.738900,43.004085,28.502708,399962,8.976120,37550.849108
510,Clatsop County,Oregon,2014.676100,1454.459378,15.780753,7.890376,38021,12.190090,28114.625523
581,Columbia County,Oregon,2406.038999,1667.098213,41.826837,27.884558,50207,12.315329,28459.688051


In [278]:
#Oregon data from the above integrated data of ACS and Covid
RDataOregon = []
#COVID total cases vs. % population in poverty
R1 = Oregondata['TotalCases'].corr(Oregondata['Poverty'])
print("\na.COVID total cases vs. % population in poverty")
print(R1)
RDataOregon.append(R1)

#COVID total deaths vs. % population in poverty
R2 = Oregondata['TotalDeaths'].corr(Oregondata['Poverty'])
print("\nb.COVID total deaths vs. % population in poverty")
print(R2)
RDataOregon.append(R2)

#COVID total cases vs. Per Capita Income level
R3 = (Oregondata['TotalCases']).corr(Oregondata['PerCapitaIncome'])
print("\nc.COVID total cases vs. Per Capita Income level")
print(R3)
RDataOregon.append(R3)

#COVID total deaths vs. Per Capita Income level
R4 = (Oregondata['TotalDeaths']).corr(Oregondata['PerCapitaIncome'])
print("\nd.COVID total deaths vs. Per Capita Income level")
print(R4)
RDataOregon.append(R4)

#COVID cases during December 2020 vs. % population in poverty
R5 = (Oregondata['Dec2020Cases']).corr(Oregondata['Poverty'])
print("\ne.COVID cases during December 2020 vs. % population in poverty")
print(R5)
RDataOregon.append(R5)

#COVID deaths during December 2020 vs. % population in poverty
R6 = (Oregondata['Dec2020Deaths']).corr(Oregondata['Poverty'])
print("\nf.COVID deaths during December 2020 vs. % population in poverty")
print(R6)
RDataOregon.append(R6)

#COVID cases during December 2020 vs. Per Capita Income level
R7 = (Oregondata['Dec2020Cases']).corr(Oregondata['PerCapitaIncome'])
print("\ng.COVID cases during December 2020 vs. Per Capita Income level")
print(R7)
RDataOregon.append(R7)

#COVID cases during December 2020 vs. Per Capita Income level
R8 = (Oregondata['Dec2020Cases']).corr(Oregondata['PerCapitaIncome'])
print("\nh.COVID cases during December 2020 vs. Per Capita Income level")
print(R8)
RDataOregon.append(R8)

scatterplotOregon = []
for i in RDataOregon:
  if i > 0.5 or i < -0.5:
    scatterplotOregon.append(i)
  else:
    continue
print("\n")
print("Values for R > 0.5 and R < -0.5")
print(scatterplotOregon)




a.COVID total cases vs. % population in poverty
0.2293950991410019

b.COVID total deaths vs. % population in poverty
0.22801173743636727

c.COVID total cases vs. Per Capita Income level
-0.2135584334770105

d.COVID total deaths vs. Per Capita Income level
-0.21351513951056433

e.COVID cases during December 2020 vs. % population in poverty
0.29487677050379707

f.COVID deaths during December 2020 vs. % population in poverty
0.386047378183474

g.COVID cases during December 2020 vs. Per Capita Income level
-0.38717063280245867

h.COVID cases during December 2020 vs. Per Capita Income level
-0.38717063280245867


Values for R > 0.5 and R < -0.5
[]


In [276]:
USAdata = []
#COVID total cases vs. % population in poverty
Rdata = integrated_df
R1 = Rdata['TotalCases'].corr(Rdata['Poverty'])
print("\na.COVID total cases vs. % population in poverty")
print(R1)
USAdata.append(R1)

#COVID total deaths vs. % population in poverty
R2 = Rdata['TotalDeaths'].corr(Rdata['Poverty'])
print("\nb.COVID total deaths vs. % population in poverty")
print(R2)
USAdata.append(R2)

#COVID total cases vs. Per Capita Income level
R3 = Rdata['TotalCases'].corr(Rdata['PerCapitaIncome'])
print("\nc.COVID total cases vs. Per Capita Income level")
print(R3)
USAdata.append(R3)

#COVID total cases vs. Per Capita Income level
R4 = Rdata['TotalCases'].corr(Rdata['PerCapitaIncome'])
print("\nd.COVID total cases vs. Per Capita Income level")
print(R4)
USAdata.append(R4)


#COVID cases during December 2020 vs. % population in poverty
R5 = Rdata['Dec2020Cases'].corr(Rdata['Poverty'])
print("\ne.COVID cases during December 2020 vs. % population in poverty")
print(R5)
USAdata.append(R5)

#COVID deaths during December 2020 vs. % population in poverty
R6 = Rdata['Dec2020Deaths'].corr(Rdata['Poverty'])
print("\nf.COVID deaths during December 2020 vs. % population in poverty")
print(R6)
USAdata.append(R6)


#COVID cases during December 2020 vs. Per Capita Income level
R7 = (Rdata['Dec2020Cases']).corr(Rdata['PerCapitaIncome'])
print("\ng.COVID cases during December 2020 vs. Per Capita Income level")
print(R7)
USAdata.append(R7)

#COVID death during December 2020 vs. Per Capita Income level
R8 = Rdata['Dec2020Deaths'].corr(Rdata['PerCapitaIncome'])
print("\nh.COVID cases during December 2020 vs. Per Capita Income level")
print(R8)
USAdata.append(R8)
scatterplot = []
for i in USAdata:
  if i > 0.5 or i < -0.5:
    scatterplot.append(i)
  else:
    continue
print(scatterplot)



a.COVID total cases vs. % population in poverty
0.03485599441173466

b.COVID total deaths vs. % population in poverty
0.04087905848836704

c.COVID total cases vs. Per Capita Income level
-0.03894663142236917

d.COVID total cases vs. Per Capita Income level
-0.03894663142236917

e.COVID cases during December 2020 vs. % population in poverty
0.07346823057542182

f.COVID deaths during December 2020 vs. % population in poverty
0.17928626120301505

g.COVID cases during December 2020 vs. Per Capita Income level
-0.16655046195992682

h.COVID cases during December 2020 vs. Per Capita Income level
-0.1856017909643021
[]
